# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)

import os
from sqlalchemy import create_engine
import re
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import FeatureUnion
import numpy as np
import pickle

import warnings

warnings.simplefilter('ignore')

In [2]:
# import libraries
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

In [3]:
# load data from database
database_filepath = "../data/DisasterResponse.db"
engine = create_engine('sqlite:///' + database_filepath)
table_name = os.path.basename(database_filepath).replace(".db","") + "_table"
df = pd.read_sql_table(table_name, engine)
df.shape

(26216, 39)

In [4]:
df.head(2)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0


In [5]:
# Extract X and y variables from the data for the modelling
X = df['message']
y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

print(f'X shape {X.shape}, y shape {y.shape}')

X shape (26216,), y shape (26216, 35)


### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    """
    Process and tokenize text for Natural Language Processing (NLP) tasks.

    Args:
        text (str): The input text string to be tokenized.

    Returns:
        list: A list of processed and tokenized words from the input text.
    """
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    words = nltk.word_tokenize(text)
    
    # remove stop words
    stopwords_ = nltk.corpus.stopwords.words("english")
    words = [word for word in words if word not in stopwords_]
    
    # extract root form of words
    words = [nltk.stem.WordNetLemmatizer().lemmatize(word, pos='v') for word in words]

    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

**Model 0: Basic Pipeline**

Components:

* `CountVectorizer` with a custom tokenizer.
* `TfidfTransformer` to transform the vectorized text into TF-IDF features.
* `MultiOutputClassifier` with `RandomForestClassifier` as the estimator.

*Purpose: This model sets up a basic text classification pipeline using Random Forest for multi-output classification.*

In [7]:
model_0 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline_fitted = model_0.fit(X_train, y_train)

In [9]:
random_message = ["I am hungry, I don't have food to eat, I don't have house I don't have clothes I count on you thank you so much"]
test_output = model_0.predict(random_message)
print(y_train.columns.values[(test_output.flatten()==1)])

['related' 'request' 'aid_related' 'food' 'direct_report']


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
def get_evaluate_multioutput(actual, predicted, col_names):
    """
    Evaluate a multi-output classification model using various metrics.

    This function calculates accuracy, precision, recall, and F1 score for
    each output label in a multi-output classification task. It then returns
    these metrics in a DataFrame.

    Args:
        actual (numpy.ndarray): Array of actual labels.
        predicted (numpy.ndarray): Array of predicted labels.
        col_names (list of str): List of column names corresponding to the
        output labels.

    Returns:
        metrics_df (pd.DataFrame): DataFrame containing accuracy, precision,
                                   recall, and F1 score for each output label.
    """
    metrics = []
    
    # Calculate evaluation metrics for each set of labels
    for i in range(len(col_names)):
        accuracy = accuracy_score(actual[:, i], predicted[:, i])
        precision = precision_score(actual[:, i], predicted[:, i], average='weighted')
        recall = recall_score(actual[:, i], predicted[:, i], average='weighted')
        f1 = f1_score(actual[:, i], predicted[:, i], average='weighted')
        
        metrics.append([accuracy, precision, recall, f1])
    
    # Create dataframe containing metrics
    metrics = np.array(metrics)
    metrics_df = pd.DataFrame(data = metrics, index = col_names, columns = ['Accuracy', 'Precision', 'Recall', 'F1'])
      
    return metrics_df    

In [11]:
# Calculate evaluation metrics for test set
y_pred = model_0.predict(X_test)
col_names = list(y.columns.values)

In [12]:
evaluate_0 = get_evaluate_multioutput(np.array(y_test), y_pred, col_names)
print(evaluate_0)

                        Accuracy  Precision    Recall        F1
related                 0.816601   0.807584  0.816601  0.801927
request                 0.893500   0.887975  0.893500  0.881535
offer                   0.994507   0.989045  0.994507  0.991768
aid_related             0.775252   0.773804  0.775252  0.773920
medical_help            0.921880   0.902609  0.921880  0.894080
medical_products        0.951480   0.943527  0.951480  0.933416
search_and_rescue       0.976198   0.965005  0.976198  0.964739
security                0.980470   0.961321  0.980470  0.970801
military                0.970095   0.959582  0.970095  0.959240
water                   0.960787   0.958051  0.960787  0.953405
food                    0.940800   0.936887  0.940800  0.935780
shelter                 0.935917   0.931427  0.935917  0.923114
clothing                0.987183   0.986380  0.987183  0.982358
money                   0.977724   0.978220  0.977724  0.967289
missing_people          0.989625   0.979

### 6. Improve your model
Use grid search to find better parameters. 

**Model 1: Grid Search with Hyperparameter Tuning**

Base Model: Uses the pipeline from Model 1.

Parameters Tuned:

* `vect__min_df`: Minimum document frequency (1 and 5).
* `tfidf__use_idf`: Whether to use IDF in TF-IDF (True and False).
* `clf__estimator__n_estimators`: Number of trees in the Random Forest (10 and 25).

Technique: `GridSearchCV` is used to find the best combination of these hyperparameters based on the f1_micro score.

*Purpose: This model aims to improve performance by tuning hyperparameters through cross-validation.*

In [13]:
parameters = {'vect__min_df': [1, 5],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[10, 25]}


cv = GridSearchCV(model_0, param_grid=parameters, scoring='f1_micro', verbose = 10)
model_gs = cv.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5; 1/8] START clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1
[CV 1/5; 1/8] END clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1;, score=nan total time=  19.5s
[CV 2/5; 1/8] START clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1
[CV 2/5; 1/8] END clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1;, score=nan total time=  20.1s
[CV 3/5; 1/8] START clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1
[CV 3/5; 1/8] END clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1;, score=nan total time=  20.3s
[CV 4/5; 1/8] START clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1
[CV 4/5; 1/8] END clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1;, score=nan total time=  19.7s
[CV 5/5; 1/8] START clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1
[CV 5/5; 1/8] END clf__

In [14]:
# Parameters for best mean test score
model_gs.best_params_

{'clf__estimator__n_estimators': 10, 'tfidf__use_idf': True, 'vect__min_df': 1}

The best results (with regard to f1_micro score) were achieved using the following parameters:

- Random Forest Classifier number of estimators = 10
- TfidfTransformer use_idf = True
- CountVectorizer minimum df = 1

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [15]:
y_pred_gs = model_gs.predict(X_test)

evaluate_1 = get_evaluate_multioutput(np.array(y_test), y_pred_gs, col_names)

print(evaluate_1)

                        Accuracy  Precision    Recall        F1
related                 0.799207   0.790445  0.799207  0.791609
request                 0.885108   0.877912  0.885108  0.870593
offer                   0.994507   0.989045  0.994507  0.991768
aid_related             0.745194   0.743742  0.745194  0.740052
medical_help            0.920049   0.895534  0.920049  0.894569
medical_products        0.949802   0.935845  0.949802  0.931667
search_and_rescue       0.976503   0.971247  0.976503  0.965480
security                0.980317   0.961318  0.980317  0.970725
military                0.969484   0.958657  0.969484  0.960508
water                   0.960330   0.955777  0.960330  0.954878
food                    0.930577   0.925575  0.930577  0.921288
shelter                 0.932560   0.926965  0.932560  0.917939
clothing                0.986573   0.985290  0.986573  0.981066
money                   0.977113   0.966336  0.977113  0.966102
missing_people          0.989625   0.979

In [16]:
# Get summary stats for first model
evaluate_0.describe()

,Accuracy,Precision,Recall,F1
count,35.000000,35.000000,35.000000,35.000000
mean,0.947169,0.936761,0.947169,0.935306
std,0.051789,0.054517,0.051789,0.056039
min,0.775252,0.773804,0.775252,0.773920
25%,0.938358,0.924095,0.938358,0.923982
50%,0.958346,0.954073,0.958346,0.949556
75%,0.981996,0.977360,0.981996,0.974916
max,0.995575,0.991170,0.995575,0.993368


In [17]:
# Get summary stats for second model
evaluate_1.describe()

,Accuracy,Precision,Recall,F1
count,35.000000,35.000000,35.000000,35.000000
mean,0.943036,0.931155,0.943036,0.931143
std,0.057865,0.060601,0.057865,0.061321
min,0.745194,0.743742,0.745194,0.740052
25%,0.932255,0.923267,0.932255,0.919613
50%,0.960330,0.953902,0.960330,0.949580
75%,0.982148,0.975150,0.982148,0.974815
max,0.995575,0.991170,0.995575,0.993368


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

**Model 2: Feature Union Pipeline**

Components:

* `FeatureUnion` to combine multiple feature extraction pipelines.
* `text_pipeline` within `FeatureUnion` includes `CountVectorizer` and `TfidfTransformer`.
* `MultiOutputClassifier` with `RandomForestClassifier` as the estimator.

*Purpose: This model allows for more complex feature engineering by using `FeatureUnion` to potentially combine text features with other types of features in the future.*

In [23]:
model_fu = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),
        ])),

        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

In [24]:
# Feature Union
pipeline_fu = model_fu.fit(X_train, y_train)

In [25]:
y_pred_fu = model_fu.predict(X_test)

evaluate_3 = get_evaluate_multioutput(np.array(y_test), y_pred_fu, col_names)

print(evaluate_3)

                        Accuracy  Precision    Recall        F1
related                 0.816143   0.807180  0.816143  0.801476
request                 0.894568   0.889607  0.894568  0.882489
offer                   0.994507   0.989045  0.994507  0.991768
aid_related             0.771437   0.769949  0.771437  0.770108
medical_help            0.920049   0.895667  0.920049  0.890467
medical_products        0.950565   0.942831  0.950565  0.930787
search_and_rescue       0.977724   0.974186  0.977724  0.968778
security                0.980470   0.961321  0.980470  0.970801
military                0.971010   0.964368  0.971010  0.959588
water                   0.961245   0.958359  0.961245  0.954277
food                    0.945529   0.942237  0.945529  0.942062
shelter                 0.935764   0.930693  0.935764  0.923301
clothing                0.986421   0.983999  0.986421  0.980968
money                   0.977418   0.972349  0.977418  0.966841
missing_people          0.989625   0.979

In [26]:
# Get summary stats for third model
evaluate_3.describe()

,Accuracy,Precision,Recall,F1
count,35.000000,35.000000,35.000000,35.000000
mean,0.947064,0.936697,0.947064,0.935085
std,0.051967,0.054056,0.051967,0.056090
min,0.771437,0.769949,0.771437,0.770108
25%,0.938129,0.923349,0.938129,0.923897
50%,0.958651,0.951237,0.958651,0.948123
75%,0.981767,0.975195,0.981767,0.973841
max,0.995575,0.991170,0.995575,0.993368


In summary, Model 0 is a straightforward pipeline for text classification. Model 1 enhances this by performing hyperparameter tuning to optimize the model. Model 2 introduces a more flexible architecture, paving the way for incorporating additional feature sets beyond text.

Based on the provided results, here's a detailed analysis to determine which model has the best performance:

**Model 0: Basic Pipeline**

* F1 Score Mean: 0.935306
* Standard Deviation: 0.056039

**Model 1: Grid Search with Hyperparameter Tuning**

* F1 Score Mean: 0.931143
* Standard Deviation: 0.061321

**Model 2: Feature Union Pipeline**

* F1 Score Mean: 0.935085
* Standard Deviation: 0.056090
  
**Analysis**

F1 Score Mean:

* Model 0 has the highest mean F1 score (0.935306), closely followed by Model 2 (0.935085).
* Model 1 has the lowest mean F1 score (0.931143).

Standard Deviation:

* Model 0 has a standard deviation of 0.056039.
* Model 2 has a slightly higher standard deviation of 0.056090.
* Model 1 has the highest standard deviation (0.061321), indicating more variability in performance.

**Conclusion**

* Best Performance: Model 0 has the best performance with the highest mean F1 score (0.935306) and a relatively low standard deviation (0.056039), indicating both strong and consistent performance.
* Second Best: Model 2 is very close in performance to Model 0, with a nearly identical mean F1 score (0.935085) and a slightly higher standard deviation (0.056090).
* Least Preferred: Model 1 has the lowest mean F1 score (0.931143) and the highest standard deviation (0.061321), making it the least preferred model in terms of both average performance and consistency.


### 9. Export your model as a pickle file

I have chosen **Model 1** as the best model for the following reasons:

* Comparable Performance: The performance metrics are very similar across all
  three models. **Model 1** achieves a mean F1 score of 0.931143 with a standard
  deviation of 0.061321, which is very close to the scores of Model 1 (mean F1
  score of 0.935306 and standard deviation of 0.056039) and Model 3 (mean F1
  score of 0.935085 and standard deviation of 0.056090). This indicates that
  **Model 1** performs at a similar level of accuracy and consistency as the other
  models.
* Grid Search Optimization: **Model 1** utilizes GridSearchCV for hyperparameter
  tuning, which systematically searches for the best combination of
  hyperparameters to optimize model performance. This thorough approach ensures
  that the model is well-tuned and can potentially achieve better performance in
  different scenarios. The use of grid search demonstrates a more rigorous and
  comprehensive method for model optimization.
  
Given the similar performance across the models and the added advantage of grid
search optimization, **Model 1** stands out as the most robust and well-tuned choice
for this project.

In [27]:
best_model = model_gs

In [31]:
# Pickle best model
pickle.dump(best_model, open('../models/disaster_model.pkl', 'wb'))

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.